# Let's use the Trapezoid Method, Simpson's Rule and Romberg Integration

In [ ]:
%matplotlib inline 
import numpy as np                    # Import numpy under alias np
import matplotlib.pyplot as plt       # Import matplotlib.pyplot under alias plt

## Define a function for taking an integral 

In [ ]:
def func(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return a*x**2 + b*x + c

## Define its integral so we know the right answer 

In [ ]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x

## Define the core of the Trapezoid Method

In [ ]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))

## Define a wrapper function to perform the Trapezoid Method

In [ ]:
def trapezoid_method(f,a,b,N):
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evaluations to use
    # note the number of chunks will be N-1
    # so if N is odd, then we don't need to adjest the last segment
    
    # Define x values to perform Trapezoid Method
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # Define the value of the integral 
    Fint = 0.0
    
    # Perform the integral using the Trapezoid Method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
     
    # Return the answer
    return Fint

## Define the core of the Simpson's Method

In [ ]:
def simpson_core(f,x,h):
    return h*( f(x) + 4*f(x+h) + f(x+2*h))/3

## Define a wrapper function to perform the Simpson's Method

In [ ]:
def simpsons_method(f,a,b,N):
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evaluations to use
    # note the number of chunks will be N-1
    # so if N is odd, then we don't need to adjest the last segment
    
    # Define x values to perform the Simpson's Method
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # Define the value of the integral 
    Fint = 0.0
    
    # If N is odd or even, we have different numbers of chunks to add
    if((N%2)==0):
        lim = 3
    else:
        lim = 2
        
    # PErform the integral using Simpson's Method
    for i in range(0,len(x)-2,2):
        Fint += simpson_core(f,x[i],h)
        print(i,i+2,x[i],x[i]+h,x[i]+2*h,x[-2],x[-1])
        
    # Apply Simpson's Method over the last interval if N is even
    if((N%2)==0):
        Fint += simpson_core(f,x[-2],0.5*h)
        
    # Return the answer     
    return Fint

## Define the Romberg Integration core

In [ ]:
def romberg_core(f,a,b,I,i):
    
    # We need the difference b-a
    h = b-a
    
    # And the increment between new function evaluations
    dh = h/2.**(i)
    
    # We need the cofactor
    K = h/2.**(i+1)
    
    # And the functon evaluations 
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    # Return the answer
    return K*M

## Define a wrapper function to perform Romberg Integration

In [ ]:
def romberg_integration(f,a,b,tol):
    
    # Define an interation variable
    i = 0 
    
    # Define a maximum number of iterations
    imax = 1000
    
    # Define am error estimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    # Set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    # Get the zeroth Romberg Iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    # Iterate by 1
    i += 1 
    
    while(delta>tol):
        
        # Find the Romberg Iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,I,i)
        
        # Compute the new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i])
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            # Iterate
            i+=1
            
            # If we've reached the maximum iterations 
            if(i>imax):
                print("Maximum iterations reached.")
                raise StopIteration('Stopping iterations after ',i)
            
    # Return the answer 
    return I[i]

In [ ]:
Answer = func_integral(1)-func_integral(0)
print(Answer)
print("Trapezoid")
print(trapezoid_method(func,0,1,100))
print("Simpson's Method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-6
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)